# Quantum Software Development Journey: 
# From Theory to Application with Classiq - Part 3

**Welcome to the Classiq Workshop Series for QClass 2024!**

In this series, we will develop the skills needed to participate in quantum software development!

- Week 1: Classiq's Basics & High-Level Functional Design
- Week 2: Using Git as a Tool for In-Team Collaboration and Open Source Contributions
- **Week 3: Introduction to Quantum Machine Learning (QML) & VQE**
- Week 4: QNN and and Advanced Applications

**[New Classiq's documentation](https://docs.classiq.io/latest/)!**




Additional resources you should use are
- The IDE of the classiq platform at [platform.classiq.io](platform.classiq.io)
- The [community Slack of Classiq](https://short.classiq.io/join-slack) - Classiq's team will answer any question you have over there, including implementation questions
- Course's [GitHub repository](https://github.com/Classiq/classiq-library/tree/main/community/QClass_2024)

  
**Good luck!**

## VQE


The Variational Quantum Eigensolver is an algorithm that finds the minimal eigenvalue of a matrix by executing a parametric circuit (also referred to as an ansatz), estimating the expected value of the matrix for the state the circuit creates (from the distribution received by the execution), and using a classical optimizer to select the next set of parameters for the circuit, until reaching convergence (or exceeding a set amount of maximum iterations).

The estimation of the expectation value is done on Pauli based matrices, so any matrix we want to perform this operation on, need to be decomposed into a sum of Pauli terms.

**In this example, we will create a simple VQE algorithm that estimates the minimal eigenvalue of the  following 2x2 matrix:**

`[[1, -1], [-1, 0]] = 1/2*I + 1/2*Z - X`

### U Gate

The single-qubit gate applies phase and rotation with three Euler angles.

Matrix representation:

$$
U(\gamma,\phi,\theta,\lambda) = e^{i\gamma}\begin{pmatrix}
\cos(\frac{\theta}{2}) & -e^{i\lambda}\sin(\frac{\theta}{2}) \\
e^{i\phi}\sin(\frac{\theta}{2}) & e^{i(\phi+\lambda)}\cos(\frac{\theta}{2}) \\
\end{pmatrix}
$$

Parameters:

- `theta`: `CReal`
- `phi`: `CReal`
- `lam`: `CReal`
- `gam`: `CReal`
- `target`: `QBit`

### VQE Implementation

In [3]:
from typing import List

from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], 
                        [            
            PauliTerm([Pauli.I], 0.5),
            PauliTerm([Pauli.Z], 0.5),
            PauliTerm([Pauli.X], -1)])


@qfunc
def main(q: Output[QBit], angles: CArray[CReal, 3]) -> None:
    allocate(1, q)
    U(angles[0], angles[1], angles[2], 0, q) 


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA, # Constrained Optimization by Linear Approximation
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)

<details>
<summary>
NOTE

</summary>

Read more about the supported optimizers:[here](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/ground-state-solving/advanced-usage/solver-customization/)
</details>


Executing from the Classiq Platform:

In [4]:
show(qprog)

Opening: https://platform.classiq.io/circuit/c44965b8-fb5a-4797-9aeb-741ef2559093?version=0.41.1


Or directly from the SDK:

In [5]:
res = execute(qprog)
# res.open_in_ide()
vqe_result = res.result()[0].value

In [6]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -0.60498046875
Optimal parameters: {'angles_0': -4.289061524925964, 'angles_1': -6.274092431830335, 'angles_2': -1.1512443092629974}
Eigenstate: {'0': (0.5426173548182918+0j), '1': (0.8399800034822258+0j)}


### Analytical Solution

- VQE is a **numerical** optimization technique used to find the ground state energy of a quantum system.
- For simple systems, an **analytical** solution can be achieved by diagonalizing the Hamiltonian matrix.

Diagonalization is possible when the Hamiltonian can be represented as a **finite-dimensional matrix** and the matrix is ״well-behaved״ (e.g., Hermitian and non-singular). 

In summary, for small systems or simple Hamiltonians, the eigenvalues and eigenvectors can be found exactly by diagonalizing the matrix. Let's show an example:

In [7]:
import numpy as np

H = np.array([[1, -1], [-1, 0]])

E , v = np.linalg.eig(H)
E_min = min(E)

v_min = np.array(v[:,np.argmin(E)])

In [8]:
print("The minimal energy energy is: ",E_min)

The minimal energy energy is:  -0.6180339887498948


**Looks like the VQE estimated the minimum energy well!**

Let's have another small validation, this time for the analytical solution of the eigenvalue equation:

$$ \hat{H}  V_{min} = E_{min} \cdot V_{min} \rightarrow V_{min} = \frac{\hat{H}  V_{min}}{E_{min}} $$

**Let's validate this relation!**

In [9]:
v_min

array([0.52573111, 0.85065081])

In [11]:
(H @ v_min)/E_min 

array([0.52573111, 0.85065081])

In [14]:
v_min == (H @ v_min)/E_min

array([False, False])

**How could it be?**

In [15]:
if np.allclose((H @ v_min)/E_min, v_min):
    print("The eigenvalue equation is validated!")
else:
    print("The eigenvalue equation is not validated!")

The eigenvalue equation is validated!


(The "`array([False, False])`" was due to rounding errors)

## Exercise - Two Qubits VQE

Now we will practice the implementation of a similar case to the last example, but this time for two qubits, following the Hamiltonian:


$H = \frac{1}{2}I \otimes I + \frac{1}{2}Z \otimes Z - X \otimes X $

Use the last example to implement and execute VQE for this Hamiltonian.

In [16]:
HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], [...]) #TODO: Complete Hamiltonian


@qfunc
def main(...) -> None:
    #TODO: Complete the function according to the instructions, choose simple ansatz.


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/518bc08d-8db8-4f62-819c-7153a22b7faf?version=0.41.1


## H₂ Molecule Problem

Now we will dive into more applicative approche of using VQE with Classiq for modeling molcoules and finding thier ground state and energy.

**Hamiltonian for $H_2$ Molecule**

The physical Hamiltonian for the hydrogen molecule $H_2$ in the [Born-Oppenheimer approximation](https://en.wikipedia.org/wiki/Born%E2%80%93Oppenheimer_approximation) can be expressed as (atomic units):

$$
\hat{H} = - \sum_{i=1}^{2} \frac{\nabla_i^2}{2} - \frac{1}{|\mathbf{r}_1 - \mathbf{R}_A|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_A|} - \frac{1}{|\mathbf{r}_1 - \mathbf{R}_B|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_B|} + \frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|} + \frac{1}{|\mathbf{R}_A - \mathbf{R}_B|}
$$

Where:
- $\mathbf{r}_1$ and $\mathbf{r}_2$ are the positions of the two electrons.
- $\mathbf{R}_A$ and $\mathbf{R}_B$ are the positions of the two protons (nuclei).
- The terms represent:
  - The kinetic energy of the electrons: $- \sum_{i=1}^{2} \frac{\nabla_i^2}{2}$
  - The electron-nucleus attractions: $- \frac{1}{|\mathbf{r}_1 - \mathbf{R}_A|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_A|} - \frac{1}{|\mathbf{r}_1 - \mathbf{R}_B|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_B|}$
  - The electron-electron repulsion: $\frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|}$
  - The nucleus-nucleus repulsion: $\frac{1}{|\mathbf{R}_A - \mathbf{R}_B|}$
 
**How can we implement such a Hamiltonian on a quantum computer?**

In the first step of this VQE process, we will need to use:
- **Jordan-Wigner Transformation**: To map the fermionic Hamiltonian to a qubit Hamiltonian.
- **Hartree-Fock Method**: To provide an initial guess for the wavefunction.
- **UCC Ansatz**: To construct the parameterized quantum circuit that will be optimized during the VQE process.


### Imports

In [48]:
from classiq import *
# from classiq import qfunc, create_model, molecule_ucc, synthesize, show, QBit, allocate, cfunc, execute, QConstant
from classiq.qmod.builtins import MoleculeProblem, Molecule, ChemistryAtom, Position
from classiq.qmod.builtins.classical_functions import molecule_problem_to_hamiltonian
from classiq.qmod.builtins.classical_execution_primitives import vqe, molecule_ground_state_solution_post_process, save
from classiq.interface.generator.expressions.enums.chemistry import Element, FermionMapping 
from classiq.interface.generator.expressions.enums import Optimizer 

### implementation

- We will define the `Molecule` structure, as well as the `MoleculeProblem` we are trying to solve.

- **The built-in functions we will use here will generate the Hamiltonian and the parameterized circuit for us.**

In [17]:
molecule_H2 = Molecule(
    spin=1,
    charge=0,
    atoms=[
        ChemistryAtom(element=Element.H, position=Position(x=0.0, y=0.0, z=0)),
        ChemistryAtom(element=Element.H, position=Position(x=0.0, y=0.0, z=0.735)), # Angstrom
    ]
)


chemistry_problem = MoleculeProblem(
    molecule=molecule_H2,
    mapping=FermionMapping.JORDAN_WIGNER,  #'BRAVYI_KITAEV'
    z2_symmetries=False, # If `z2_symmetries=False`, 4 qubits need to be allocated in main
    freeze_core=True,
    remove_orbitals=[]
)

molecule_problem = QConstant("molecule_problem", MoleculeProblem, chemistry_problem)

@qfunc
def main():
    q = QArray("q")
    allocate(4,q) # allocate 4 qubits if `z2_symmetries=False`, else: allocate 1 qubit
    molecule_hartree_fock(molecule_problem,q)
    molecule_ucc(molecule_problem = molecule_problem,excitations=[1,2],qbv=q)


@cfunc
def cmain():
    ham = molecule_problem_to_hamiltonian(molecule_problem)
    vqe_result = vqe(
          hamiltonian=(ham),
          maximize=False,
          initial_point=[],
          optimizer=Optimizer.COBYLA, 
          max_iteration=100,
          tolerance=0.001,
          step_size=0,
          skip_compute_variance=False,
          alpha_cvar=1.0)
    molecule_result = molecule_ground_state_solution_post_process(molecule_problem,vqe_result)
    save({"vqe_res": vqe_result, "ham": ham, 'molecule_result': molecule_result}) 


In [18]:
qmod = create_model(main,classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/20721c4e-d449-4ee1-8aa4-7497ec8b20a5?version=0.41.1


In [19]:
execution = execute(qprog)
res = execution.result()
execution.open_in_ide()

In [20]:
type(res)

list

In [21]:
len(res)

3

In [22]:
vqe_result = res[0].value  
hamiltonian = res[1].value
total_energy = res[2].value['total_energy'] # nuclear_repulsion_energy + vqe_energy
nuclear_energy = res[2].value['nuclear_repulsion_energy']

In [23]:
print(f"Optimal energy: {vqe_result.energy} Ha") # vqe_energy WITHOUT nuclear_repulsion_energy
print(f"Nuclear repulsion energy: {nuclear_energy} Ha")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.857821819076969 Ha
Nuclear repulsion energy: 0.7199689944489797 Ha
Optimal parameters: {'param_0': -3.1874228460829723, 'param_1': 0.013694712771728136, 'param_2': 0.11708980402134109}
Eigenstate: {'0110': (0.038273277230987154+0j), '1010': (0.08838834764831845+0j), '0101': (0.9953505192895616+0j)}


In [56]:
hamiltonian # pauli:[0,1,2,3] == [I,X,Y,Z]

[{'pauli': [0, 0, 0, 0], 'coefficient': -0.8105479805373275},
 {'pauli': [0, 0, 0, 3], 'coefficient': 0.17218393261915557},
 {'pauli': [0, 0, 3, 0], 'coefficient': -0.2257534922240239},
 {'pauli': [0, 3, 0, 0], 'coefficient': 0.17218393261915554},
 {'pauli': [3, 0, 0, 0], 'coefficient': -0.22575349222402397},
 {'pauli': [0, 0, 3, 3], 'coefficient': 0.12091263261776627},
 {'pauli': [0, 3, 0, 3], 'coefficient': 0.16892753870087907},
 {'pauli': [2, 2, 2, 2], 'coefficient': 0.04523279994605784},
 {'pauli': [1, 1, 2, 2], 'coefficient': 0.04523279994605784},
 {'pauli': [2, 2, 1, 1], 'coefficient': 0.04523279994605784},
 {'pauli': [1, 1, 1, 1], 'coefficient': 0.04523279994605784},
 {'pauli': [3, 0, 0, 3], 'coefficient': 0.16614543256382414},
 {'pauli': [0, 3, 3, 0], 'coefficient': 0.16614543256382414},
 {'pauli': [3, 0, 3, 0], 'coefficient': 0.17464343068300445},
 {'pauli': [3, 3, 0, 0], 'coefficient': 0.12091263261776627}]

**Finally, the total energy of the hydrogen molecule is estimated as:**

In [25]:
print(f"The total energy is: {total_energy} Ha")

The total energy is: -1.1378528246279895 Ha


In [27]:
total_energy == vqe_result.energy + nuclear_energy

True

what is the value by theory?

**In the third homework assignment, you will compare and validate these results by using 2 qubits Hamiltonian!**

## Read More


- [**VQE Method for Molecule Energy Solver**](https://docs.classiq.io/latest/explore/built_in_apps/chemistry/chemistry/#2-constructing-and-synthesizing-a-ground-state-solver)
  - This link provides a detailed demonstration of how to use the `construct_chemistry_model` function, which constructs a VQE model for Molecule eigensolver.

- [**Combinatorial Optimization using QAOA**](https://docs.classiq.io/latest/explore/applications/optimization/electric_grid_optimization/electric_grid_optimization/)
  - This link covers the use of the Quantum Approximate Optimization Algorithm (QAOA) for combinatorial optimization problems, with a specific example on Electric Grid Optimization.



## Solution - Two Qubits VQE

In [ ]:
HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], 
                        [            
            PauliTerm([Pauli.I, Pauli.I], 0.5),
            PauliTerm([Pauli.Z, Pauli.Z], 0.5),
            PauliTerm([Pauli.X, Pauli.X], -1)
                        ])


@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
# show(qprog)